# Çözücüler ⚙️

Bu egzersizde, farklı `çözücülerin` `LogisticRegression` modelleri üzerindeki etkilerini araştıracaksınız.

👇 Veri kümesini içe aktarmak için aşağıdaki kodu çalıştırın

In [1]:
import pandas as pd

df = pd.read_csv("https://d32aokrjazspmn.cloudfront.net/materials/solvers_dataset.csv")
df.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol,quality rating
0,9.47,5.97,7.36,10.17,6.84,9.15,9.78,9.52,10.34,8.80,6
1,10.05,8.84,9.76,8.38,10.15,6.91,9.70,9.01,9.23,8.80,7
2,10.59,10.71,10.84,10.97,9.03,10.42,11.46,11.25,11.34,9.06,4
3,11.00,8.44,8.32,9.65,7.87,10.92,6.97,11.07,10.66,8.89,8
4,12.12,13.44,10.35,9.95,11.09,9.38,10.22,9.04,7.68,11.38,3


- Veri kümesi farklı şaraplardan oluşmaktadır 🍷
- Özellikler şarapların farklı niteliklerini tanımlar 
- Hedef 🎯 bir uzman tarafından verilen kalite değerlendirmesidir

## 1. Hedef mühendisliği

Bu bölümde, değerlendirmeleri ikili bir hedefe dönüştüreceksiniz.

👇 Her değerlendirme için kaç gözlem bulunmaktadır?

In [2]:
df['quality rating'].unique()

array([ 6,  7,  4,  8,  3,  1,  2, 10,  5,  9])

In [3]:
df['quality rating'].value_counts()

quality rating
10    10143
5     10124
1     10090
2     10030
8      9977
6      9961
9      9955
7      9954
4      9928
3      9838
Name: count, dtype: int64

❓ Hedefi ikili sınıflandırma görevine dönüştürerek `y` oluşturun, burada 6'nın altındaki kalite değerlendirmeleri kötü [0], 6 ve üzeri değerlendirmeler iyi [1] olacak

In [5]:
y = df['quality rating'].map(lambda x: 0 if x < 6 else 1)
print(y.head())


0    1
1    1
2    0
3    1
4    0
Name: quality rating, dtype: int64


❓ Yeni ikili hedefin sınıf dengesini kontrol edin

In [6]:
print(y.value_counts())
print("\nSınıf oranları:")
print(y.value_counts(normalize=True))


quality rating
0    50010
1    49990
Name: count, dtype: int64

Sınıf oranları:
quality rating
0    0.5001
1    0.4999
Name: proportion, dtype: float64


❓ Özellikleri normalleştirerek `X`'inizi oluşturun. Bu farklı çözücülerin adil karşılaştırılmasına olanak sağlayacaktır.

In [7]:
from sklearn.preprocessing import MinMaxScaler
X = df.drop(columns=['quality rating'])
scaler = MinMaxScaler().fit(X)
X_scaled = scaler.transform(X)

X_scaled = pd.DataFrame(X_scaled)
X_scaled.columns = X.columns


## 2. LogisticRegression çözücüleri

❓ Lojistik Regresyon modelleri farklı **çözücüler** kullanılarak optimize edilebilir. Mevcut çözücülerin karşılaştırmasını yapın:
- Uyum süresi - hangi çözücü **en hızlı**?
- Kesinlik - kesinlik puanları **ne kadar farklı**?

Lojistik Regresyon için mevcut çözücüler: `['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']`
 
Bu 5 çözücü hakkında daha fazla bilgi için [bu Stack Overflow konusuna](https://stackoverflow.com/questions/38640109/logistic-regression-python-solvers-defintions) göz atın

In [8]:
from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression

solvers = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

scores = []
fit_times = []

for solver in solvers:

    cv_log_s = cross_validate(LogisticRegression(solver=solver), X_scaled, y, cv = 5, scoring = ['precision'])

    scores.append(cv_log_s['test_precision'].mean())
    fit_times.append(cv_log_s['fit_time'].mean())

solvers_performance = pd.DataFrame({"precision score":scores, "fit time": fit_times}, index = solvers)
solvers_performance

,precision score,fit time
newton-cg,0.874383,0.057590
lbfgs,0.874200,0.042249
liblinear,0.874449,0.062050
sag,0.874340,0.170938
saga,0.874386,0.302758


In [9]:
fastest_solver = solvers_performance['fit time'].idxmin()
fastest_solver

'lbfgs'

<details>
    <summary>ℹ️ Yorumumuz için buraya tıklayın</summary>

Maliyet fonksiyonumuz 5 çözücünün de bulduğu global bir minimuma sahip olacak kadar "kolay" olduğundan, tüm çözücüler benzer kesinlik puanları üretmelidir. Derin Öğrenme'de olduğu gibi çok karmaşık maliyet fonksiyonları için, farklı çözücüler kayıp fonksiyonunun farklı değerlerinde durabilir.

**Şarap veri kümesi**
    
Mevcut veri kümesinde sklearn'in <a href="https://scikit-learn.org/stable/modules/generated/sklearn.inspection.permutation_importance.html">permutation_importance</a> ile özellik önemini kontrol ederseniz, birçok özelliğin neredeyse 0 önemine sahip olduğunu göreceksiniz. Liblinear çözücü, bir defada sadece *bir* yön boyunca hareket eder ve diğerlerini L1 düzenlileştirme ile düzenler (yani, beta değerlerini 0'a ayarlar), bu da birçok özelliğin hedefi tahmin etmede o kadar da önemli olmadığı bir veri kümesi için iyi bir uyum sağlayabilir.

❗️En iyi çözücüyü arama maliyeti vardır. Varsayılanla (`lbfgs`) devam etmek genel olarak en çok zaman tasarrufu sağlayabilir, sklearn başlamak için hangi çözücüyü seçeceğiniz konusunda fikir vermek için bu tabloyu sunar: 

<img src="https://wagon-public-datasets.s3.amazonaws.com/05-Machine-Learning/04-Under-the-Hood/solvers-chart.png" width=700>

</details>

###  🧪 Kodunuzu test edin

In [10]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'solvers',
    fastest_solver=fastest_solver
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S16D4-S-data-solvers/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 1 item

test_solvers.py::TestSolvers::test_fastest_solver PASSED                 [100%]

============================== 1 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/solvers.pickle

git commit -m 'Completed solvers step'

git push origin master



## 3. Stokastik Gradyan İnişi

Lojistik Regresyon modelleri ayrıca Stokastik Gradyan İnişi ile de optimize edilebilir.

❓ **Stokastik Gradyan İnişi** ile optimize edilmiş bir Lojistik Regresyon modelini değerlendirin. Kesinlik puanı ve eğitim süresi 2. bölümde eğitilen modellerin performansı ile nasıl karşılaştırılır?

<details>
<summary>💡 İpucu</summary>

- Takılırsanız, [SGDClassifier belgelerine](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) bakın!

</details>

In [13]:
from sklearn.linear_model import SGDClassifier

sgd_model = SGDClassifier(loss="log_loss")

cv_sgd = cross_validate(sgd_model, X_scaled, y, cv = 5, scoring = 'precision')
print(cv_sgd['test_score'].mean())
print(cv_sgd['fit_time'].mean())

0.8701571831329087
0.06955299377441407


☝️ SGD modeli, benzer performans için en kısa sürelerden birine sahip olmalıdır (hatta `liblinear`'dan bile daha kısa olabilir). Bu, Gradyan İnişinin her dönemini aynı anda 100k satırı belleğe yüklemek yerine tek bir satırda gerçekleştirmenin doğrudan bir etkisidir.

## 4. Tahminler

❓ En iyi modeli (kısa uyum süresi ve yüksek kesinlik ile dengelenen) kullanarak aşağıdaki şarabın ikili kalitesini (0 veya 1) tahmin edin. Şunları kaydedin:
- `predicted_class`
- `predicted_proba_of_class` (yani modeliniz 1 sınıfını tahmin ettiyse, 1'in sınıf olması gerektiğine inanma olasılığı nedir, 0 ile 1 arasında olmalıdır)

In [14]:
new_wine = pd.read_csv('https://d32aokrjazspmn.cloudfront.net/materials/solvers_new_wine.csv')
new_wine

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,sulphates,alcohol
0,9.54,13.5,12.35,8.78,14.72,9.06,9.67,10.15,11.17,12.17


In [17]:
best_model = SGDClassifier(loss="log_loss").fit(X_scaled,y)

new_X = pd.DataFrame(scaler.transform(new_wine))
new_X.columns = new_wine.columns

best_model.predict(new_X)
predicted_class = best_model.predict(new_X)[0]
print(predicted_class)
predicted_proba_of_class = best_model.predict_proba(new_X)[0, predicted_class]
print(predicted_proba_of_class)

0
0.9589696948968034


# 🏁  Kodunuzu kontrol edin ve notebook'unuzu gönderin

In [18]:
from nbresult import ChallengeResult

result = ChallengeResult(
    'new_data_prediction',
    predicted_class=predicted_class,
    predicted_proba_of_class=predicted_proba_of_class
)
result.write()
print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/dilay/.pyenv/versions/workintech/bin/python
cachedir: .pytest_cache
rootdir: /Users/dilay/Desktop/S16D4-S-data-solvers/tests
plugins: dash-3.3.0, anyio-4.8.0, typeguard-4.4.2
collecting ... collected 2 items

test_new_data_prediction.py::TestNewDataPrediction::test_predicted_class PASSED [ 50%]
test_new_data_prediction.py::TestNewDataPrediction::test_predicted_proba PASSED [100%]

============================== 2 passed in 0.10s ===============================


💯 You can commit your code:

git add tests/new_data_prediction.pickle

git commit -m 'Completed new_data_prediction step'

git push origin master

